In [ ]:
'''Script for checking delensed Blens spectra, using precaulculated Btemplates as input.
'''
import os, sys
from os.path import join as opj
import hashlib
import argparse

import numpy as np
import healpy as hp
from astropy.io import fits

import plancklens
from plancklens.helpers import mpi
from plancklens import utils

from plancklens.sims import planck2018_sims
from lerepi.data.dc08 import data_08d as sims_if

from component_separation.MSC.MSC import pospace as ps

mpi.barrier = lambda : 1 # redefining the barrier
ioreco_edges = np.array([2, 30, 200, 300, 500, 700, 1000, 1500, 2000, 3000, 4000, 5000])
cmbs4_edges = np.array([2, 30, 60, 90, 120, 150, 180, 200, 300, 500, 700, 1000, 1500, 2000, 3000, 4000, 5000])

edges_center = (edges[1:]+edges[:-1])/2.
sims_May  = sims_if.ILC_May2022(fg)

nside = 2048
lmax_cl = 2048
lmax_lib = 3*lmax_cl-1

beam = 2.3
lmax_transf = 4000
transf = hp.gauss_beam(beam / 180. / 60. * np.pi, lmax=lmax_transf)

cls_path = opj(os.path.dirname(plancklens.__file__), 'data', 'cls')
cls_len = utils.camb_clfile(opj(cls_path, 'FFP10_wdipole_lensedCls.dat'))
clg_templ = cls_len['ee']
clc_templ = cls_len['bb']

clg_templ[0] = 1e-32
clg_templ[1] = 1e-32

sha_edges = hashlib.sha256()
sha_edges.update((str(edges)+str(blm_suffix)).encode())

nlevels = [1.2, 1.5, 1.7, 2., 2.3, 5., 10., 50.]
dirid = sha_edges.hexdigest()[:4]

In [ ]:
'''
Most important settings
'''
dirroot = '/global/cscratch1/sd/sebibel/cmbs4/s08d/cILC_%2s_plotdata/'%(fg)
if not(os.path.isdir(dirroot)):
    os.mkdir(dirroot)
if not(os.path.isdir(dirroot + '{}'.format(dirid))):
    os.mkdir(dirroot + '{}'.format(dirid))
    
simid_lower = 0
simid_upper = 1
fg = '00'
blm_suffix = ''
    
if args.edges == 'bk14':
    edges = bk14_edges
elif args.edges == 'ioreco':
    edges = ioreco_edges
elif args.edges == 'cmbs4':
    edges = cmbs4_edges
    
simids = np.arange(simid_lower, simid_upper)

In [ ]:
def getfn_blm_lensc(simidx, fg, it):
    if blm_suffix == '':
        rootstr = '/global/cscratch1/sd/sebibel/cmbs4/s08b/cILC_%s_test/iterator_p_p_%04d_OBD/'%(fg, simidx)
        return rootstr + 'ffi_p_it%s%s/blm%s_%04d_it%d.npy'%(it, simidx, it)
blm_templ = getfn_blm_lensc(0, '00', 4)

In [ ]:
"""
This calculates for each noiselevel and each fg sim-ds, QE and MAP delensed blensing and bpower spectra.
TODO: add bwf delensed
"""
lib_nm = dict()
bcl_L_nm, bcl_cs_nm, bwfcl_cs_nm = np.zeros(shape=(len(nlevels),len(edges))), np.zeros(shape=(len(nlevels),len(edges))), np.zeros(shape=(len(nlevels),len(edges)))
outputdata = np.zeros(shape=(6,len(nlevels),len(edges)-1))

for nlev in nlevels:
    nlev_mask = sims_if.get_nlev_mask(nlev)
    lib_nm.update({nlev: ps.map2cl_binned(nlev_mask, clc_templ[:lmax_lib], edges, lmax_lib)})

for simid in simids[mpi.rank::mpi.size]:
    for nlevi, nlev in enumerate(nlevels):
        nlev_mask = sims_if.get_nlev_mask(nlev)     
        # bcl_cs_nm = lib_nm[nlev].map2cl(bmap_cs_buff)
        blm_L_buff = hp.almxfl(utils.alm_copy(planck2018_sims.cmb_len_ffp10.get_sim_blm(simid), lmax=lmax_cl), transf)
        bmap_L_buff = hp.alm2map(blm_L_buff, nside)
        bcl_L_nm = lib_nm[nlev].map2cl(bmap_L_buff)
        if blm_suffix == '':
            blm_lensc_MAP_buff = hp.read_alm(getfn_blm_lensc(simid, fg, 12))
            bmap_lensc_MAP_buff = hp.alm2map(blm_lensc_MAP_buff, nside=nside)
            # blm_lensc_QE_buff = np.load(getfn_blm_lensc(simid, fg, 0))
            # bmap_lensc_QE_buff = hp.alm2map(blm_lensc_QE_buff, nside=nside)
        else:
            blm_lensc_MAP_buff = np.load(getfn_blm_lensc(simid, fg, 12))
            bmap_lensc_MAP_buff = hp.alm2map(blm_lensc_MAP_buff, nside=nside)
            # blm_lensc_QE_buff = np.load(getfn_blm_lensc(simid, fg, 0))
            # bmap_lensc_QE_buff = hp.alm2map(blm_lensc_QE_buff, nside=nside)

        bcl_Llensc_MAP_nm = lib_nm[nlev].map2cl(bmap_L_buff-bmap_lensc_MAP_buff)    
        bcl_Llensc_QE_nm = lib_nm[nlev].map2cl(bmap_L_buff-bmap_lensc_QE_buff)

        outputdata[0][nlevi] = bcl_L_nm
        outputdata[1][nlevi] = np.zeroes_like(bcl_L_nm) # bcl_cs_nm

        outputdata[2][nlevi] = bcl_Llensc_MAP_nm
        outputdata[3][nlevi] = np.zeroes_like(bcl_Llensc_MAP_nm) # bcl_cslensc_MAP_nm  

        outputdata[4][nlevi] = np.zeroes_like(bcl_Llensc_MAP_nm) # bcl_Llensc_QE_nm           
        outputdata[5][nlevi] = np.zeroes_like(bcl_Llensc_QE_nm) # bcl_cslensc_QE_nm

    # np.save(dirroot + '{}'.format(dirid) + '/Lenscarf_plotdata_ClBBwf_sim%04d_fg%2s_res2b3acm.npy'%(simid, fg), outputdata)
    print('mpi.rank {} nlev {} mask: {} / {} done.'.format(mpi.rank, str(nlev), simid+1, len(simids)))

In [ ]:
hp.mollview(outputdata[0])
hp.mollview(outputdata[2])